In [2]:
import pandas as pd

cols = [
    'PRODUCTID',
    'PRODUCTNDC',
    'NDCPACKAGECODE',
    'PACKAGEDESCRIPTION',
]

package = pd.read_csv('./package.txt', sep='\t') \
            .loc[:, cols]

cols = [
    'PRODUCTID',
    'PROPRIETARYNAME',
    'PROPRIETARYNAMESUFFIX',
    'NONPROPRIETARYNAME',
    'DOSAGEFORMNAME',
    'SUBSTANCENAME',
    'ACTIVE_NUMERATOR_STRENGTH',
    'ACTIVE_INGRED_UNIT',
    'DEASCHEDULE'
]

product = pd.read_csv('./product.txt', sep='\t') \
            .loc[:, cols]

fda = pd.merge(left=package, right=product, how='left', on='PRODUCTID', suffixes=(False,False))

fda.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 209111 entries, 0 to 209110
Data columns (total 12 columns):
 #   Column                     Non-Null Count   Dtype 
---  ------                     --------------   ----- 
 0   PRODUCTID                  209111 non-null  object
 1   PRODUCTNDC                 209111 non-null  object
 2   NDCPACKAGECODE             209111 non-null  object
 3   PACKAGEDESCRIPTION         209111 non-null  object
 4   PROPRIETARYNAME            58022 non-null   object
 5   PROPRIETARYNAMESUFFIX      5801 non-null    object
 6   NONPROPRIETARYNAME         58022 non-null   object
 7   DOSAGEFORMNAME             58022 non-null   object
 8   SUBSTANCENAME              56831 non-null   object
 9   ACTIVE_NUMERATOR_STRENGTH  56831 non-null   object
 10  ACTIVE_INGRED_UNIT         56831 non-null   object
 11  DEASCHEDULE                3113 non-null    object
dtypes: object(12)
memory usage: 20.7+ MB


In [3]:
def process_str_unit(ele):
    if pd.isna(ele):
        return []

    elmod = [x.replace('/1','').strip() for x in ele.split(';')]

    # edge case
    if '' in elmod:
        elmod = [x for x in elmod if x != '']

    res = []
    for x in elmod:
        if '.' in x:
            if x[0]=='.':
                res.append(x.replace('.','0.'))
            elif x.split('.')[0][-1].isnumeric():
                res.append(x)
            else:
                res.append(x.replace('.','0.'))
        else:
            res.append(x)

    return res

fda['Strength_List'] = fda['ACTIVE_NUMERATOR_STRENGTH'].apply(process_str_unit)
fda['Unit_List'] = fda['ACTIVE_INGRED_UNIT'].apply(process_str_unit)

In [4]:
import numpy as np

def build_strength(row):
    MAX_LEN = 3

    stl = row['Strength_List']
    unl = row['Unit_List']

    if len(stl) > MAX_LEN and len(unl) > MAX_LEN:
        return np.nan

    if len(stl) != len(unl):
        print(f"DERP!:({len(stl)}) ",end='')
        print(stl)
        print(f"DORP!:({len(unl)})",end='')
        print(unl)
        return

    str_unit = [f'{stl[i]} {unl[i]}' for i,_ in enumerate(stl)]

    return ', '.join(str_unit)

fda['Strength'] = fda.apply(build_strength, axis=1)

In [5]:
def get_str(s):
    return '' if pd.isna(s) else s.strip()

def get_package_desc(s):
    return '' if pd.isna(s) else s.split('>')[-1].strip().split('(')[0].strip().split(',')[0].strip()

def build_repl_part(s):
    return

def build_repl(row):
    prop_name = get_str(row['PROPRIETARYNAME'])
    prop_suff = get_str(row['PROPRIETARYNAMESUFFIX'])
    desc = get_package_desc(row['PACKAGEDESCRIPTION'])
    strn = get_str(row['Strength'])
    gen_name = get_str(row['NONPROPRIETARYNAME'])
    dea = get_str(row['DEASCHEDULE'])

    gen_part = f'{gen_name.title()} ' if gen_name != '' else ''
    prop_part = (f'({prop_name.title()} {prop_suff.title()}) ' if prop_suff != '' else f'({prop_name.title()}) ') if prop_name != '' else ''
    strn_part = f'{strn} ' if strn != '' else ''
    dea_part = f'(Schedule {dea}) ' if dea != '' else ''
    desc_part = f'[{desc.title()}]' if desc != '' else ''

    return f'{gen_part}{prop_part}{strn_part}{dea_part}{desc_part}'

fda['repl'] = fda.apply(build_repl, axis=1)

fda['repl']

0         Diluent (Sterile Diluent) 1 mL/mL [10 Ml In 1 ...
1         Florbetapir F 18 (Amyvid) 51 mCi/mL [30 Ml In ...
2         Florbetapir F 18 (Amyvid) 51 mCi/mL [50 Ml In ...
3         Flortaucipir F-18 (Tauvid) 51 mCi/mL [30 Ml In...
4         Flortaucipir F-18 (Tauvid) 51 mCi/mL [50 Ml In...
                                ...                        
209106                                    [120 G In 1 Tube]
209107                                     [30 G In 1 Tube]
209108                                     [60 G In 1 Tube]
209109                                      [2 G In 1 Tube]
209110                                     [60 G In 1 Tube]
Name: repl, Length: 209111, dtype: object

In [6]:
fda['repl'].iloc[0:20].apply(lambda x: print(x))

Diluent (Sterile Diluent) 1 mL/mL [10 Ml In 1 Vial]
Florbetapir F 18 (Amyvid) 51 mCi/mL [30 Ml In 1 Vial]
Florbetapir F 18 (Amyvid) 51 mCi/mL [50 Ml In 1 Vial]
Flortaucipir F-18 (Tauvid) 51 mCi/mL [30 Ml In 1 Vial]
Flortaucipir F-18 (Tauvid) 51 mCi/mL [50 Ml In 1 Vial]
Dulaglutide (Trulicity) 0.75 mg/0.5mL [.5 Ml In 1 Syringe]
Dulaglutide (Trulicity) 0.75 mg/0.5mL [.5 Ml In 1 Syringe]
Dulaglutide (Trulicity) 1.5 mg/0.5mL [.5 Ml In 1 Syringe]
Dulaglutide (Trulicity) 1.5 mg/0.5mL [.5 Ml In 1 Syringe]
Galcanezumab (Emgality) 120 mg/mL [1 Ml In 1 Syringe]
Galcanezumab (Emgality) 120 mg/mL [1 Ml In 1 Syringe]
Ixekizumab (Taltz) 80 mg/mL [1 Ml In 1 Syringe]
Ixekizumab (Taltz) 80 mg/mL [1 Ml In 1 Syringe]
Ixekizumab (Taltz) 80 mg/mL [1 Ml In 1 Syringe]
Ixekizumab (Taltz) 80 mg/mL [1 Ml In 1 Syringe]
Galcanezumab (Emgality) 120 mg/mL [1 Ml In 1 Syringe]
Selpercatinib (Retevmo) 80 mg [120 Capsule In 1 Bottle]
Selpercatinib (Retevmo) 80 mg [60 Capsule In 1 Bottle]
Galcanezumab (Emgality) 100 mg/

0     None
1     None
2     None
3     None
4     None
5     None
6     None
7     None
8     None
9     None
10    None
11    None
12    None
13    None
14    None
15    None
16    None
17    None
18    None
19    None
Name: repl, dtype: object

In [7]:
fda.to_csv('./FDA.csv')
